<a href="https://colab.research.google.com/github/Reksy747/DameMitaiMeme/blob/master/Dame_Mitai_MEME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dame Mitai MEME**

Demo for paper "First Order Motion Model for Image Animation"
Original Code:
https://github.com/AliaksandrSiarohin/first-order-model
https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb

**BUAT FOLDER 'first-order-motion-model' di google drive mu
dan upload video dan gambar yang akan di buat deepfake ke folder 'first-order-motion-model' yang telah di buat tadi**

**Video source untuk deepfake DAME MITAI :** https://anonfiles.com/Pb2b7dJfoe/dameyo_mp4

**Tentukan nama file gambar dan driving video (sesuaikan dengan video yang anda upload ke gdrive di folder 'first-order-motion-model')**

In [ ]:
DRIVING_VIDEO = "dameyo.mp4"
GAMBAR_TARGET = "target.png"
OUTPUT = "DameMitai_MEME.mp4"

**CLONE REPO DULS**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 236, done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 236
Receiving objects: 100% (236/236), 71.46 MiB | 30.30 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [ ]:
cd /content/first-order-model

**COPY DATA vox-adv-cpk.pth.tar dan vox-cpk.pth.tar ke folder 'first-order-motion-model' yang telah di buat tadi
filenya ada disini :** https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw

**Mount your Google drive folder on Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**Load driving video and source image**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/{}'.format(GAMBAR_TARGET))
driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/{}'.format(DRIVING_VIDEO), memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('/content/gdrive/My Drive/first-order-motion-model/{}'.format(OUTPUT), [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from drive folder

HTML(display(source_image, driving_video, predictions).to_html5_video())